In [21]:
import os 
from dotenv import load_dotenv
load_dotenv()

openai_api_key=os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4.1-2025-04-14")

In [22]:
llm.invoke('hi')

AIMessage(content='Hello! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-CvOepy2YWtzQcyQiuhCiubXJ0ybFe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b98d2-dc43-7743-b7a4-38723bf19d7f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn    
warnings.filterwarnings('ignore')

In [7]:
from langchain_community.document_loaders import PyPDFLoader

In [9]:
loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf")
pdf_data = loader.load()

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=50,chunk_overlap=20)
document =text_splitter.split_documents(pdf_data)
document

[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='* corresponding author - jkim72@kent.edu'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='Revolutionizing Mental Health Care through'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': 

In [14]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()

In [15]:
from langchain_community.vectorstores import Chroma
vectordb =Chroma.from_documents(documents=document,embedding=embedding)

In [16]:
retriever =vectordb.as_retriever()

In [52]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

In [53]:
query='What does the paper say about langchain?'

Set logging for the queries.

In [54]:
import logging

logging.basicConfig()
logging.getLogger("langchain_classic.retrievers.multi_query").setLevel(logging.INFO)

In [55]:
docs = retriever.invoke('query')
docs

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['1. What information is available about the term "query"?  ', '2. Can you provide documents related to queries or search queries?  ', '3. What are the definitions and uses of "query" in different contexts?']


[Document(metadata={'total_pages': 6, 'moddate': '2023-12-31T03:52:06+00:00', 'creationdate': '2023-12-31T03:50:13+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf', 'creator': 'Microsoft Word', 'page': 2, 'producer': 'PyPDF', 'author': 'IEEE', 'page_label': '3'}, page_content='2. How state is queried ?'),
 Document(metadata={'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf', 'page_label': '2', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'page': 1, 'author': 'IEEE', 'producer': 'PyPDF', 'creationdate': '2023-12-31T03:50:13+00:00', 'creator': 'Microsoft Word', 'total_pages': 6}, page_content='the message. It can be a user query, a system'),
 Document(metadata={'page': 3, 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'page_label': '4', 'total_pages': 6, 'author': 'IEE

From the log results, you can see that the LLM generated three additional queries from different perspectives based on the given query.
The returned results are the union of the results from each query.

Self-Querying Retriever

In [73]:
from langchain_classic.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.documents import Document
from langchain_classic.chains.query_constructor.base import AttributeInfo
import lark

In [74]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

In [75]:
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

In [76]:
vectordb = Chroma.from_documents(docs, embedding)

In [ ]:
document_content_description = "Brief summary of a movie."

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
)